# Demo Notebook:
## Random Survival Forest

In [1]:
import os
from pathlib import Path
import sys
node_type = os.getenv('BB_CPU')
venv_dir = f'/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-{node_type}'
venv_site_pkgs = Path(venv_dir) / 'lib' / f'python{sys.version_info.major}.{sys.version_info.minor}' / 'site-packages'
if venv_site_pkgs.exists():
    sys.path.insert(0, str(venv_site_pkgs))
    print(f"Added path '{venv_site_pkgs}' at start of search paths.")
else:
    print(f"Path '{venv_site_pkgs}' not found. Check that it exists and/or that it exists for node-type '{node_type}'.")

%load_ext autoreload
%autoreload 2

Added path '/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-icelake/lib/python3.10/site-packages' at start of search paths.


In [2]:
import pytorch_lightning
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import sqlite3
from dataclasses import dataclass
import logging
from FastEHR.dataloader import FoundationalDataModule
import pickle
from tqdm import tqdm

from pycox.datasets import support
from pycox.evaluation import EvalSurv
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from torch.utils.data import TensorDataset, DataLoader

from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sksurv.datasets import load_gbsg2
from sksurv.preprocessing import OneHotEncoder
from sksurv.ensemble import RandomSurvivalForest

torch.manual_seed(1337)
logging.basicConfig(level=logging.INFO)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = "cpu"    # if more informative debugging statements are needed
print(f"Using device: {device}.")

Using device: cuda.


In [3]:
set_config(display="text")  # displays text representation of estimators

# Load data

In [4]:
def get_dataloaders(dataset, competing_risk, sample_size=None, seed=None):

    match dataset.lower():
        case "pycox":
            df_train = support.read_df()
            df_test = df_train.sample(frac=0.2)
            df_train = df_train.drop(df_test.index)
            df_val = df_train.sample(frac=0.2)
            df_train = df_train.drop(df_val.index)
            
            cols_standardize = ['x0', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13']
            cols_leave = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6']
            
            standardize = [([col], StandardScaler()) for col in cols_standardize]
            leave = [(col, None) for col in cols_leave]
            
            x_mapper = DataFrameMapper(standardize + leave)
            
            x_train = x_mapper.fit_transform(df_train).astype('float32')
            x_val = x_mapper.transform(df_val).astype('float32')
            x_test = x_mapper.transform(df_test).astype('float32')
            
            get_target = lambda df: (df['duration'].values, df['event'].values)
            y_train = get_target(df_train)
            y_val = get_target(df_val)
            y_test = get_target(df_test)
            
            t_train, e_train = y_train
            t_val, e_val = y_val
            t_test, e_test = y_test
            
            t_train_max = np.amax(t_train)
            t_train = t_train / t_train_max
            t_val = t_val / t_train_max
            t_test = t_test / t_train_max
            
    
        case "hypertension" | "cvd":
    
            # Training samples
            if sample_size is not None:
                save_path =  f"/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_{dataset}/" + f"benchmark_data/N={sample_size}_seed{seed}.pickle" 
            else:
                save_path = f"/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_{dataset}/" + "benchmark_data/all.pickle"
                
            with open(save_path, "rb") as handle:
                print(f"Loading training dataset from {save_path}")
                data_train = pickle.load(handle)
            
            # display(data["X_train"].head())
            # display(data["y_train"])
            # print(data.keys())
            
            data = {}
            data["X_train"] = data_train["X_train"]
            data["y_train"] = data_train["y_train"]
    
            # Test and validation samples
    
            save_path = f"/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_{dataset}/" + "benchmark_data/all.pickle"
            with open(save_path, "rb") as handle:
                print(f"Loading validation/test datasets from {save_path}")
                data_val_test = pickle.load(handle)
                
            data["X_val"] = data_val_test["X_val"]
            data["y_val"] = data_val_test["y_val"]
            data["X_test"] = data_val_test["X_test"]
            data["y_test"] = data_val_test["y_test"]
    
            # Convert to correct formats
            x_train = data["X_train"].to_numpy(dtype=np.float32)
            x_val = data["X_val"].to_numpy(dtype=np.float32)
            x_test = data["X_test"].to_numpy(dtype=np.float32)
            
            t_train = np.asarray([i[1] for i in data["y_train"]])
            t_val = np.asarray([i[1] for i in data["y_val"]])        
            t_test = np.asarray([i[1] for i in data["y_test"]])
    
            if competing_risk is False:
                e_train = np.asarray([0 if i[0] == 0 else 1 for i in data["y_train"]])
                e_val = np.asarray([0 if i[0] == 0 else 1 for i in data["y_val"]])
                e_test = np.asarray([0 if i[0] == 0 else 1 for i in data["y_test"]])
            else:
                e_train = np.asarray([i[0] for i in data["y_train"]])
                e_val = np.asarray([i[0] for i in data["y_val"]])
                e_test = np.asarray([i[0] for i in data["y_test"]])

    # display(x_train.shape)
    # display(type(x_train))
    # display(type(x_train[0,0]))
    # display(e_train.shape)
    # display(type(e_train))
    # display(type(e_train[0]))
    # display(t_train.shape)
    # display(type(t_train))
    # display(type(t_train[0]))
    # print(np.mean(e_train))
    # print(np.mean(t_train))
    # print(np.std(t_train))
    # print(np.mean(x_train))
    # print(t_train.min())
    # print(t_train.max())
    # print(np.unique(e_test, return_counts=True))

    # print(x_train.shape)
    # print(t_train.shape)
    # print(e_train.shape)

    
    Xtrain = pd.DataFrame(x_train, )    #  columns=list(dm.train_set.tokenizer._stoi.keys())[1:]
    Xval = pd.DataFrame(x_val, )    #  columns=list(dm.train_set.tokenizer._stoi.keys())[1:]
    Xtest = pd.DataFrame(x_test, )    #  columns=list(dm.train_set.tokenizer._stoi.keys())[1:]

    if competing_risk is False:
        ytrain = np.array([(_yk, _yt) for _yk, _yt in zip(e_train, t_train)], dtype=[('cens', 'bool'), ('time', '<f8')])
        yval = np.array([(_yk, _yt) for _yk, _yt in zip(e_val, t_val)], dtype=[('cens', 'bool'), ('time', '<f8')])
        ytest = np.array([(_yk, _yt) for _yk, _yt in zip(e_test, t_test)], dtype=[('cens', 'bool'), ('time', '<f8')])
    else:
        # Package does not support Competing Risks
        raise NotImplementedError
        
        # ytrain = np.array([(_yk, _yt) for _yk, _yt in zip(e_train, t_train)])# , dtype=[('cens', 'float'), ('time', '<f8')])
        # yval = np.array([(_yk, _yt) for _yk, _yt in zip(e_val, t_val)]) #, dtype=[('cens', 'float'), ('time', '<f8')])
        # ytest = np.array([(_yk, _yt) for _yk, _yt in zip(e_test, t_test)]) #, dtype=[('cens', 'float'), ('time', '<f8')])
    # print(Xtrain.head())
    # print(ytrain[:5])

    return (Xtrain, ytrain), (Xval, yval), (Xtest, ytest)




# Example dataloader function usage

In [5]:
dataset_train, dataset_val, dataset_test = get_dataloaders("Hypertension", False, sample_size=2999, seed=1)

Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=2999_seed1.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle


In [6]:
print(dataset_test[0].head())
print(dataset_test[1][:3])

   0    1    2    3    4    5    6    7    8    9    ...  269  270  271  272  \
0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
1  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  1.0  0.0  1.0  1.0   
2  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0  1.0  1.0  1.0   
3  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  1.0  0.0  1.0  1.0   
4  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  1.0  0.0  0.0  0.0   

   273  274  275  276  277  278  
0  0.0  0.0  0.0  0.0  0.0  0.0  
1  1.0  1.0  1.0  1.0  1.0  1.0  
2  0.0  1.0  1.0  1.0  0.0  0.0  
3  0.0  1.0  1.0  1.0  0.0  0.0  
4  0.0  1.0  1.0  1.0  0.0  0.0  

[5 rows x 279 columns]
[( True, 1.11616468) (False, 0.34082222) (False, 2.12383461)]


# Train model

In [9]:
dataset = "Hypertension"
competing_risk = False
sample_sizes = [int(np.exp(_log_n)) for _log_n in np.linspace(np.log(3000), np.log(500000), 10)]      # [3000, 12500, 30000, 60000, 100000]: # 600, 1200, 
# sample_sizes = [None]
sample_sizes = [None]
print(sample_sizes)

# the time grid which we generate over
t_eval = np.linspace(0, 1, 1000) 
# the time grid which we calculate scores over
time_grid = np.linspace(start=0, stop=1 , num=300)


[None]


In [10]:
model_names, all_ctd, all_ibs, all_inbll = [], [], [], []

for sample_size in sample_sizes:

    seeds = [1,2,3,4,5]

    for seed in seeds:
        # Load dataset
        dataset_train, dataset_val, dataset_test = get_dataloaders(dataset, competing_risk, sample_size=sample_size, seed=seed)

        # Create RSF model with default bootstrap values due to memory constraints
        print(f"Fitting Random Survival Forest")
        model_name = f"RandomSurvivalForest-{'CR' if competing_risk else 'SR'}-{dataset}-Ns{sample_size}-seed{seed}"

        rsf = RandomSurvivalForest(
            bootstrap=True,
            max_samples=1000,    
            random_state=seed,
            low_memory=False
        )

        # Train model
        rsf.fit(dataset_train[0], dataset_train[1])
    
        # Test
        bsz = 512
        print(f"Evaluating performance by splitting {dataset_test[0].shape} test samples into batches of size {bsz}")
        
        ctd = []
        ibs = []
        inbll = []
        for batch_idx in range(0, dataset_test[0].shape[0], bsz):
        
            batch_dataset_test = (dataset_test[0][batch_idx:batch_idx + bsz], dataset_test[1][batch_idx:batch_idx + bsz])
            actual_bsz = batch_dataset_test[0].shape[0]
        
            # Predict survival functionfor batch
            surv = rsf.predict_survival_function(batch_dataset_test[0], return_array=True)
            
            # Find the indices in rsf.unique_times_ that are closest to values in t_eval, so we can evaluate the RSF if the same way as other benchmarks
            closest_indices = [np.abs(rsf.unique_times_ - v).argmin() for v in t_eval]
            surv_reduced = surv[:, closest_indices]
        
            # Format appropriately
            df_surv = pd.DataFrame(np.transpose(surv_reduced), index=t_eval)
            
            lbls_test = np.zeros((actual_bsz,))
            t_test = np.zeros((actual_bsz,))
            for sample_idx in range(actual_bsz):
                lbls_test[sample_idx] = 1 if batch_dataset_test[1][sample_idx][0] == True else 0
                t_test[sample_idx] = batch_dataset_test[1][sample_idx][1]
        
            # Same treatment as in SurvivEHR
            ev = EvalSurv(df_surv, t_test, lbls_test, censor_surv='km')
            ctd.append(ev.concordance_td())
            ibs.append(ev.integrated_brier_score(time_grid))
            inbll.append(ev.integrated_nbll(time_grid))
        
            # print(f"Scores up to sample {batch_idx+bsz}:".ljust(50) + f"Ctd: {np.mean(ctd):.3f}. IBS: {np.mean(ibs):.4f}. INBLL: {np.mean(inbll):.3f}")
        
        ctd = np.mean(ctd)
        ibs = np.mean(ibs)
        inbll = np.mean(inbll)
        
        print(f"{model_name}:".ljust(20) + f"N={sample_size}.".ljust(15) + f"Ctd: {ctd}. IBS: {ibs}. INBLL: {inbll}")
        model_names.append(model_name)
        all_ctd.append(ctd)
        all_ibs.append(ibs)
        all_inbll.append(inbll)

        # print(f"\tRandom Survival Forest ({'CR' if competing_risk else 'SR'}):".ljust(20) + f"N={sample_size}.".ljust(15) + f"Ctd: {ctd}. IBS: {ibs}. INBLL: {inbll}")





Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-NsNone-seed1:N=None.        Ctd: 0.7334417601219098. IBS: 0.08537406119428995. INBLL: 0.28391626643192946
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples in

# Output across different setups

Hypertension Single Risk

Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=2999_seed1.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns2999-seed1:N=2999.        Ctd: 0.7282302075112793. IBS: 0.08621154954731365. INBLL: 0.28871543149135453
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=2999_seed2.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns2999-seed2:N=2999.        Ctd: 0.7176099102933101. IBS: 0.08614562139045558. INBLL: 0.2887419993400637
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=2999_seed3.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns2999-seed3:N=2999.        Ctd: 0.725162206467073. IBS: 0.08637657949416157. INBLL: 0.2910756044788604
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=2999_seed4.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns2999-seed4:N=2999.        Ctd: 0.7253000661534662. IBS: 0.08638218294695646. INBLL: 0.2886658753281553
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=2999_seed5.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns2999-seed5:N=2999.        Ctd: 0.7073145691175673. IBS: 0.08677313026091858. INBLL: 0.29295831074259987
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=5296_seed1.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns5296-seed1:N=5296.        Ctd: 0.7307664669072022. IBS: 0.08598138446312305. INBLL: 0.2866990790335502
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=5296_seed2.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns5296-seed2:N=5296.        Ctd: 0.7212394819716147. IBS: 0.0862328463914893. INBLL: 0.28886788628817617
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=5296_seed3.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns5296-seed3:N=5296.        Ctd: 0.7309289866498436. IBS: 0.0856623569108884. INBLL: 0.28648448686977207
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=5296_seed4.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns5296-seed4:N=5296.        Ctd: 0.7312435752279859. IBS: 0.08590582215276027. INBLL: 0.28684876792506386
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=5296_seed5.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns5296-seed5:N=5296.        Ctd: 0.7203162233603995. IBS: 0.08604122476770122. INBLL: 0.28825727892812936
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=9351_seed1.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns9351-seed1:N=9351.        Ctd: 0.7334370607124504. IBS: 0.08591820468383374. INBLL: 0.2871082931819225
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=9351_seed2.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns9351-seed2:N=9351.        Ctd: 0.7238028585502345. IBS: 0.08623695996789647. INBLL: 0.2893503368066942
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=9351_seed3.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns9351-seed3:N=9351.        Ctd: 0.7328232046080633. IBS: 0.08568325050406676. INBLL: 0.2855522913210286
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=9351_seed4.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns9351-seed4:N=9351.        Ctd: 0.7379233781582918. IBS: 0.08532761274209975. INBLL: 0.2843510055560878
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=9351_seed5.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns9351-seed5:N=9351.        Ctd: 0.7254458099126424. IBS: 0.08589642402445634. INBLL: 0.2874225624574327
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=16509_seed1.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns16509-seed1:N=16509.       Ctd: 0.7332183151169774. IBS: 0.0857744372532512. INBLL: 0.2881193081506518
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=16509_seed2.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns16509-seed2:N=16509.       Ctd: 0.7259535675715743. IBS: 0.08599681704020067. INBLL: 0.2881870638463769
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=16509_seed3.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns16509-seed3:N=16509.       Ctd: 0.7335647225487043. IBS: 0.08569165619394287. INBLL: 0.286653513373421
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=16509_seed4.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns16509-seed4:N=16509.       Ctd: 0.7352914637605852. IBS: 0.08567037809057747. INBLL: 0.28622495650168056
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=16509_seed5.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns16509-seed5:N=16509.       Ctd: 0.7261746400691371. IBS: 0.08590718259578324. INBLL: 0.287704729107248
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=29148_seed1.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns29148-seed1:N=29148.       Ctd: 0.7352184844361275. IBS: 0.08565569366742493. INBLL: 0.2860001717468009
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=29148_seed2.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns29148-seed2:N=29148.       Ctd: 0.7308598287348472. IBS: 0.0857986054299634. INBLL: 0.2870386533982125
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=29148_seed3.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns29148-seed3:N=29148.       Ctd: 0.7321740832481968. IBS: 0.08583405165621479. INBLL: 0.28707580562214857
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=29148_seed4.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns29148-seed4:N=29148.       Ctd: 0.7322737985570358. IBS: 0.0856785357081943. INBLL: 0.2856072772133489
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=29148_seed5.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns29148-seed5:N=29148.       Ctd: 0.7351927975936952. IBS: 0.08543992926188197. INBLL: 0.28525550889373735
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=51461_seed1.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns51461-seed1:N=51461.       Ctd: 0.7378650920478877. IBS: 0.08535166446999269. INBLL: 0.2847189165157441
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=51461_seed2.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns51461-seed2:N=51461.       Ctd: 0.732137231380254. IBS: 0.08588334230004704. INBLL: 0.2873841929792508
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=51461_seed3.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns51461-seed3:N=51461.       Ctd: 0.7263991649458978. IBS: 0.08599177853566699. INBLL: 0.2891157037173396
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=51461_seed4.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns51461-seed4:N=51461.       Ctd: 0.7312668565018567. IBS: 0.08565831828362806. INBLL: 0.2863501196174393
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=51461_seed5.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns51461-seed5:N=51461.       Ctd: 0.7319395187596088. IBS: 0.08577892898741978. INBLL: 0.2869413901920903
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=90856_seed1.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns90856-seed1:N=90856.       Ctd: 0.7366405504648993. IBS: 0.08541940988952396. INBLL: 0.284310737330521
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=90856_seed2.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns90856-seed2:N=90856.       Ctd: 0.7293940611590456. IBS: 0.08606738201995517. INBLL: 0.2885921317687771
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=90856_seed3.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns90856-seed3:N=90856.       Ctd: 0.7276041798315561. IBS: 0.0858220724757242. INBLL: 0.28696782067989235
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=90856_seed4.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns90856-seed4:N=90856.       Ctd: 0.7282552052084719. IBS: 0.0857053255994171. INBLL: 0.28654757337063896
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=90856_seed5.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns90856-seed5:N=90856.       Ctd: 0.729633837470379. IBS: 0.08609941604636591. INBLL: 0.288474775013163
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=160407_seed1.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns160407-seed1:N=160407.      Ctd: 0.7350462033676957. IBS: 0.0855525193711657. INBLL: 0.2856431711007079
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=160407_seed2.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns160407-seed2:N=160407.      Ctd: 0.730794091304328. IBS: 0.08579409033065355. INBLL: 0.28656278871877316
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=160407_seed3.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns160407-seed3:N=160407.      Ctd: 0.734944275106826. IBS: 0.08554384727543793. INBLL: 0.28523767613817136
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=160407_seed4.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns160407-seed4:N=160407.      Ctd: 0.7330660186636005. IBS: 0.08557438024279117. INBLL: 0.28594566948650546
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=160407_seed5.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns160407-seed5:N=160407.      Ctd: 0.7266529262953968. IBS: 0.08577937003206611. INBLL: 0.2868966917591112
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=283203_seed1.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns283203-seed1:N=283203.      Ctd: 0.7349307506933636. IBS: 0.08536186043163121. INBLL: 0.28442764999506404
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=283203_seed2.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns283203-seed2:N=283203.      Ctd: 0.72937449106186. IBS: 0.08605281736849169. INBLL: 0.2882972231576968
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=283203_seed3.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns283203-seed3:N=283203.      Ctd: 0.7298699789933027. IBS: 0.08559917203641122. INBLL: 0.28580538784702414
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=283203_seed4.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns283203-seed4:N=283203.      Ctd: 0.7328236706454961. IBS: 0.08577893775244719. INBLL: 0.2865074799194559
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/N=283203_seed5.pickle
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
Fitting Random Survival Forest
Evaluating performance by splitting (35758, 279) test samples into batches of size 512
RandomSurvivalForest-SR-Hypertension-Ns283203-seed5:N=283203.      Ctd: 0.7282869967785536. IBS: 0.08600833577523895. INBLL: 0.2882276822872275